<a href="https://colab.research.google.com/github/Seyisha/Seyisha/blob/main/kowope_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 71 kB/s 


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb

In [12]:
train=pd.read_csv("/content/Train_exp.csv")

In [13]:
test=pd.read_csv("/content/Test_exp.csv")

In [ ]:
class Func():
    def __init__(self,train,label, test, model, model_type,random_type):
        self.train,self.label,self.test=train,label,test
        self.model,self.model_type=model,model_type
        self.random_type=random_type
      
        assert  model_type in ("catboost","xgboost","lightgbm"), "incorrect model_type"

    def __call__(self, plot=true):
        return self.fit(plot)

    def fit(self,plot):
        def catboost_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)], early_stopping_round =500,
                           verbose=50,use_best_model=True)
            x_test_predict=self.model.predict.proba(X_test)[:,1]
            x_train_predict=self.model.predict.proba(X_train)[:,1]
            self.val_p[test_index]=x_test_predict
            self.test_p += self.model.predict.proba(self.test)[:,1]
            return x_test_predict,x_train_predict


        def xgboost_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_round=30,
                           verbose= True, eval_metric="auc")
            x_test_predict=self.model.predict.proba(X_test,ntree_limit=self.model.get_booster().best_ntree_limit)[:,1]
            x_train_predict=self.model.predict.proba(X_train,ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            self.val_p[test_index]=x_test_predict
            self.test_p += self.model.predict.proba(self.test, num_iteration = self.model.best_iteration_)[:,1]
            return x_test_predict,x_train_predict


        def lightgbm_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train, eval_set=[(X_test,y_test)],verbose=True,
                           eval_metric ="auc",early_stopping_round=30 )
            x_test_predict=self.model.predict.proba

        

        



